In [82]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [83]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.loop

In [ ]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
# Local modules

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

### Why do we need this?
We have identified at least 3 different kinds of loops already:

    Learner (training)
    Source/Gym (Data Access)
    Agent (How an AI takes in data, generates actions)

### What is a loop?

    It should be capable of containing inner loops. 
    It should be able to handle "phases" that might be similar to each other. 
    It should self-describe its structure. 
    It should be easy to know which parts of the loop are taking long/short amounts of time.
    It should be flexible in state modification.
    It should alternatively make it easy show what fields are being changed at what points in time.

In [203]:
# export
def grab_order(m):
    sig=inspect.signature(m)
    return -1 if '_order' not in sig.parameters else sig.parameters['_order'].default 

def ishook(name):
    return not name.startswith('_') and \
          (name.startswith('on_') or name.startswith('after_') or \
          name.startswith('before_') or name.startswith('failed_') or \
          name.startswith('finally_'))

class Loop(GetAttr):
    _default='_base'
    indexes=None
    
    def __init__(self,
                 inner_loops:Optional[List['Loop']]=None, # Internal `Loop` objects that are called on `indexes` 
                 cbs:Optional[List['Callback']]=None, # Callbacks to be used for a given `Loop`. These will also be used in internal loops.
                 persist_cbs:bool=False, # Whether to review callbacks once a loop ends.
                 base_loop:Optional['Loop']=None, # The highest level loop.
                 indexes:Optional[List[int]]=None #indexes (found from `self.hooks`)
                ):
        store_attr(but='cbs,inner_loops,indexes')
        self.indexes=ifnone(indexes,self.indexes)
        self.cbs=L()
        self.inner_loops=L()
        self.add_objs(L(cbs),self.add_cb)
        self.add_objs(L(inner_loops),self.add_loop)
        self._base=None
        self.loop_sequence=None
        
    def _grab_objs(self,obj_cls,ls): 
        return L(o for o in ls if isinstance(o,obj_cls))
        
    def add_loop(self, loop):
        "Instantiate, add to `self.loops`"
        if isinstance(loop, type):   loop=loop(base_loop=self)
        elif loop.base_loop is None: loop.base_loop=ifnone(self.base_loop,self)
        self.inner_loops.append(loop)
        return self

    def remove_loop(self, loop):
        "Instantiate, remove from `self.loops`"
        loop.base_loop=None
        if isinstance(loop, type): 
            self.remove_objs(self._grab_objs(loop,self.inner_loops),self.remove_loop)
        else:
            if loop in self.inner_loops: self.inner_loops.remove(loop)
        return self
        
    def add_objs(self,objs,adder):
        "add `cbs` to `self`"
        L(objs).map(adder)
        return self

    def remove_objs(self,objs,remover):
        "rm all `objs` from `self` using `remover`"
        L(objs).map(remover)
        return self
        
    def add_cb(self, cb):
        "Instantiate, set as field in self, set as field in self._default, and add to `self.cbs`"
        if isinstance(cb,type): cb=cb()
        setattr(cb,self._default,self)
        setattr(self,cb.name,cb)
        self.cbs.append(cb)
        return self

    def remove_cb(self, cb):
        "Instantiate, rm `cb` from self, rm `cb` from self._default, and remove from `self.cbs`"
        if isinstance(cb,type): 
            self.remove_objs(self._grab_objs(cb,self.cbs),self.remove_cb)
        else:
            setattr(cb,self._default,None)
            if hasattr(self, cb.name): delattr(self, cb.name)
            if cb in self.cbs: self.cbs.remove(cb)
        return self
    
    @classmethod
    def hooks(cls):
        hooks=[hook for k,hook in inspect.getmembers(cls) if ishook(k)]
        # Might be able to simplify python 3.7+
        return {str(i):hook for i,hook in enumerate(sorted(hooks,key=grab_order))}

In [204]:
class Outer(Loop):
    def before_step(self):           return None
    def on_step(self,_order=2):      return None
    def after_step(self,_order=3):   return None
    def failed_step(self,_order=4):  return None
    def finally_step(self,_order=5): return None

class Inner(Loop):
    def before_iteration(self):  return None
    def on_iteration(self):      return None
    def after_iteration(self):   return None
    def failed_iteration(self):  return None
    def finally_iteration(self): return None

loop=Outer(inner_loops=Inner())

In [205]:
loop.hooks()

{'0': <function __main__.Outer.before_step(self)>,
 '1': <function __main__.Outer.on_step(self, _order=2)>,
 '2': <function __main__.Outer.after_step(self, _order=3)>,
 '3': <function __main__.Outer.failed_step(self, _order=4)>,
 '4': <function __main__.Outer.finally_step(self, _order=5)>}

In [18]:
# export
class CustomLoop(Loop):pass

In [ ]:
# export 
class CustomCallback()

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 02_fastai.loop.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 06f_memory.tensorboard.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/02_fastai.loop.ipynb
